**First you will need to download the example dataset by executing the cell below:**

In [ ]:
print("Downloding the example dataset ... please wait ... ")
![ ! -d "FaceRecognition" ] && wget "FaceRecognition.zip" "https://dl.dropboxusercontent.com/s/q030cu844joczm6/FaceRecognition.zip"
![ ! -d "FaceRecognition" ] && unzip -q ~/hands-on/FaceRecognition.zip -d ~/hands-on && rm ~/hands-on/FaceRecognition.zip 
print("Dataset downloaded, carry on!")

# First Level Analysis with Nilearn

[Nilearn](https://nilearn.github.io/stable/index.html) `GLM stats` module allows fast and easy MRI statistical analysis. It leverages `Nibabel` and other `Python` libraries.


## Accessing the preprocessed (fMRIprep) data

BIDS applications, such as `fMRIprep`, output data into a data structure similar to `BIDS` organization principals. And these data can also be inspected using [PyBIDS](https://bids-standard.github.io/pybids/index.html).


In [ ]:
import os
from bids.layout import BIDSLayout

ds_path = 'FaceRecognition/data/bids'

# Initialize the BIDS layout and include the derivatives in it
layout = BIDSLayout(ds_path, derivatives = True)

With the `PyBIDS` we can easlily find the proprocessed files that we'd need for the analysis. Let's get the `sub-04` **preprocessed** anatomical and functional `MNI152NLin2009cAsym` space image files. 

In [ ]:
sID = '04'
anat = layout.get(subject=sID, datatype='anat', space='MNI152NLin2009cAsym', desc='preproc', extension='.nii.gz', \
                  return_type='filename')
print('Subject''s', sID, 'preprocessed anatomical image:')
print(*anat, sep='\n')

bold = layout.get(subject=sID, datatype='func', space='MNI152NLin2009cAsym', desc='preproc', extension='.nii.gz', \
                 return_type='filename')
print('\nSubject''s', sID, 'preprocessed functional images:')
print(*bold, sep='\n')

## Model components

A GLM model has an **outcome variable** (the BOLD signal/our MRI images) and **predictors** (Events, Confounds).

### MRI images
We need to specify which images we want to analyse. 

Here we will analyse `sub-04` 9 functional runs. We will also specify the subject's anatomical image (warped to the standard space) to use it as a background image when plotting results. 

### Events

We also need to specify the events that were happaning during the functional acquisitions. The events files are stored in the `func` foder and we can find them with `PyBIDS`.

In [ ]:
events = layout.get(subject=sID, datatype='func', suffix='events', extension=".tsv", return_type='filename')
print('Subject''s', sID, 'event files:')
print(*events, sep='\n')

Let's display the first rows of events of the first run.

In [ ]:
import pandas as pd # pandas is a library for data manipulation 
events_run1 = pd.read_table(events[0])
events_run1.head()

### Confounds

Usually, we also want to include confounds computed during preprocessing (e.g., motion artifacts) as regressors of no interest. The confounds are computed by `fmriprep` and can be found in `derivatives/fmriprep/{sub}/func/` directory.

Let's find these files with `PyBIDS`.

In [ ]:
confounds = layout.get(subject=sID, datatype='func', desc='confounds', extension=".tsv", return_type='filename')
print('Subject''s', sID, 'confound files:')
print(*confounds, sep='\n')

Let's get a list of all confound names.

In [ ]:
confounds_run1 = pd.read_table(confounds[0])
list(confounds_run1)

In [ ]:
# Total number of confounds created:
len(list(confounds_run1))

The fMRIPrep pipeline generates a large array of possible confounds. The most well established confounding variables in neuroimaging are the six head-motion parameters (three rotations and three translations) - the common output of the head-motion correction (also known as realignment) of popular fMRI preprocessing software such as SPM or FSL. Let's include them in our model. 

Let's display these confounds of the first run.

In [ ]:
confounds_of_interest = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']
confounds_run1[confounds_of_interest]

Now we need to get the confounds of interest of all runs. These will be included in our `GLM` design. We will create a list of confound tables (9 tables). 

In [ ]:
confounds_glm = []
for conf_file in confounds:
    this_conf = pd.read_table(conf_file)
    conf_subset = this_conf[confounds_of_interest].fillna(0) # replace NaN with 0
    confounds_glm.append(conf_subset)
    
print(type(confounds_glm))
print(len(confounds_glm))

## Performing the GLM analysis

It is now time to create and estimate a ``FirstLevelModel`` object, that will generate the *design matrix*.



### Creating the First Level Model

There are a lot of important parameters one needs to define within a `FirstLevelModel` and the majority of them will have a prominent influence on your results. [Check the documentation!](https://nilearn.github.io/stable/modules/generated/nilearn.glm.first_level.FirstLevelModel.html)

For the model specification, we need the TR of the functional images, luckily we can extract this information with `PyBIDS`.

In [ ]:
# Get the TR
TR = layout.get_tr()
print('TR:', TR)

Now we can specify the model with the parameters of our choice. Here we will specify the folowing:
* **t_r**, of course
* **slice_time_ref**: *This parameter indicates the time of the reference slice used in the slice timing preprocessing step of the experimental runs. It is expressed as a percentage of the t_r (time repetition), so it can have values between 0. and 1. Default=0.* Let's see if we get this information from our fMRIPrep Methods. 
* **hrf_model**: defines the HRF model to be used.
    * 

        ‘spm’: This is the HRF model used in SPM. See nilearn.glm.first_level.spm_hrf.

        ‘spm + derivative’: SPM model plus its time derivative. This gives 2 regressors. See nilearn.glm.first_level.spm_hrf, and nilearn.glm.first_level.spm_time_derivative.

        ‘spm + derivative + dispersion’: Idem, plus dispersion derivative. This gives 3 regressors. See nilearn.glm.first_level.spm_hrf, nilearn.glm.first_level.spm_time_derivative, and nilearn.glm.first_level.spm_dispersion_derivative.

        ‘glover’: This corresponds to the Glover HRF. See nilearn.glm.first_level.glover_hrf.

        ‘glover + derivative’: The Glover HRF + time derivative. This gives 2 regressors. See nilearn.glm.first_level.glover_hrf, and nilearn.glm.first_level.glover_time_derivative.

        ‘glover + derivative + dispersion’: Idem, plus dispersion derivative. This gives 3 regressors. See nilearn.glm.first_level.glover_hrf, nilearn.glm.first_level.glover_time_derivative, and nilearn.glm.first_level.glover_dispersion_derivative.

        ‘fir’: Finite impulse response basis. This is a set of delayed dirac models.

* **drift_model**: specifies the desired drift model for the design matrices. It can be ‘polynomial’, ‘cosine’ or None. Default=’cosine’.
* **high_pass**: specifies the cut frequency of the high-pass filter in Hz for the design matrices. Used only if drift_model is ‘cosine’. Default=0.01 (1/128, as in SPM).
* **smoothing_fwhm**: the full-width at half maximum in millimeters of the spatial smoothing to apply to the signal (smoothing was not done in fMRIPrep!).
* **noise_model**: {‘ar1’, ‘ols’} The temporal variance model. Default=’ar1’.





In [ ]:
from nilearn.glm.first_level import FirstLevelModel
fmri_glm = FirstLevelModel(
    t_r = TR,
    slice_time_ref = TR/2,
    hrf_model = 'spm',
    drift_model = 'Cosine',
    high_pass = 1./128,
    smoothing_fwhm = 6,
    noise_model = 'ar1'
)

### Fitting the model

Now that we have specified the model, we can run it on our specified data. We need to include the list of our functional image files (we named that `bold`), the list of events timing files (we named that `events`), and the list of our confound tables (one per run; which we named `confounds_glm`).

In [ ]:
fmri_glm = fmri_glm.fit(bold, events, confounds_glm)

### Inspecting the Design Matrix

We can now inspect the design matrix of our `GLM` model (rows represent time, and columns contain the predictors).

In [ ]:
design_matrices = fmri_glm.design_matrices_

The `design_matrices` is a list of 9 tables (one per run). Let's look at the first run's design matrix.

In [ ]:
design_matrices[0]

From the design matrix, we can extract and plot the expected signal of our conditions. Here we will plot it for the first run.

In [ ]:
dm = design_matrices[0][['FAMOUS', 'UNFAMILIAR']]
dm.columns.name = 'Condition'
dm.index.name = 'Seconds'
dm.plot(figsize=(12,4), title='Expected responses per condition')

**Q: How was this expected signal obtained?**

In [ ]:
# let's see how the modelled noise regressors look
dm = design_matrices[0][['drift_1','drift_2', 'drift_3', 'drift_4']]
dm.columns.name = 'Regressor'
dm.index.name = 'Seconds'
dm.plot(figsize=(12,4), title='Expected signal')

We can also plot the actual design matrix (here again for the first run).

In [ ]:
from nilearn.plotting import plot_design_matrix
plot_design_matrix(design_matrices[0], output_file=None)

import matplotlib.pyplot as plt
fig = plt.gcf()
fig.set_size_inches(8,6)
plt.show()

### Detecting voxels with significant effects
Now we will compute fixed effects of the 9 runs and generate related images.

#### Contrast specification
If we have the same number of regressors in each run, we can specify the contrast only for one run and it would automatically be reused for other runs.

However, in this dataset we can't do that. One of the subjects have one regressor less (the drift parameter) in one of the runs (because this run was shorter than the other runs; 170 vs 208 volumes).

Therefore, I here create contrasts for each run separately and add them to a list. 

In [ ]:
contrast_list = []
import numpy as np
for design_matrix in design_matrices:
    
    """A small routine to append zeros in contrast vectors"""
    n_columns = design_matrix.shape[1] #number of predictors in our model
    def pad_vector(contrast_, n_columns):    
        return np.hstack((contrast_, np.zeros(n_columns - len(contrast_))))
    
    contrasts = {'Famous_Unfamiliar': pad_vector([1, 0, 0, -1], n_columns),
                'Unfamiliar_Famous': pad_vector([-1, 0, 0, 1], n_columns),
                'Faces_Scrambled': pad_vector([1, 0, -2, 1], n_columns),
                'Scrambled_Faces': pad_vector([-1, 0, 2, -1], n_columns),
                 'EffectsOfInterest': np.eye(n_columns)[[0,2,3]]}
    
    contrast_list.append(contrasts)

Let’s look at the contrasts for the first run.

In [ ]:
from nilearn.plotting import plot_contrast_matrix
for key, values in contrast_list[0].items():
    plot_contrast_matrix(values, design_matrix=design_matrices[0])
    plt.suptitle(key)

plt.show()

#### Computing result maps

You can compute the `effect size` maps, `t-statistics` maps, `z-scores` and some other types. See the [documentation](https://nilearn.github.io/dev/modules/generated/nilearn.glm.Contrast.html) for more information.

Below, we compute the estimated effect. It is in BOLD signal unit, but has no statistical guarantees, because it does not take into account the associated variance.

In [ ]:
eff_map = fmri_glm.compute_contrast([c['Faces_Scrambled'] for c in contrast_list],
                                    output_type='effect_size')

In order to get statistical significance, we form a `t-statistic`, and directly convert is into `z-scale`. The `z-scale` means that the values are scaled to match a standard Gaussian distribution (mean=`0`, variance=`1`), across voxels, if there were now effects in the data. (To get a `t-map` instead, you'd need to specify `output_type='stat'`.)

In [ ]:
z_map = fmri_glm.compute_contrast([c['Faces_Scrambled'] for c in contrast_list],
                                  output_type='z_score')

### Plotting thresholded maps

#### An arbitrary z threshold

Let's display the thresholded z-score map on top of the subject's anatomical image. Let's use arbitrarily a threshold of `3.0` in `z-scale`. We'll see later how to use corrected thresholds. 

See the `plot_stat_map` [documentation](https://nilearn.github.io/dev/modules/generated/nilearn.plotting.plot_stat_map.html) for all the possible parameters you can specify to the plot.

In [ ]:
from nilearn.plotting import plot_stat_map, plot_glass_brain

plot_stat_map(
    z_map, 
    bg_img = anat[0], 
    threshold=  3.0,
    display_mode = 'ortho', black_bg=True,
    title =' Faces > Scrambled (Z>3)'
)
plt.show()

We can also plot our maps as glass (trasnparent) brain. 

In [ ]:
plot_glass_brain(z_map, threshold = 3.0, black_bg = True, plot_abs = False,
                 title='Faces > Scrambled (Z>3)')
plt.show()

### Statistical signifiance testing
One should worry about the statistical validity of the procedure: here we used an arbitrary threshold of 3.0 but the threshold should provide some guarantees on the risk of false detections (aka `type-1` errors in statistics). One
first suggestion is to **control the false positive rate** (`fpr`) at a certain level, e.g. `p < .001`.

#### Control the false positive rate

In [ ]:
# Obtain the statistical threshold
from nilearn.glm.thresholding import threshold_stats_img

_, threshold = threshold_stats_img(z_map, alpha = .001, height_control = 'fpr')

print('Uncorrected p<.001 threshold: %.3f' % threshold)

# plot the thresholded map
plot_stat_map(z_map, bg_img = anat[0], threshold = threshold,
              display_mode = 'ortho', black_bg = True,
              title = 'Faces > Scrambled  (p<.001, Uncorrected)')
plt.show()

#### Family Wise Error (FWE) correction

The example above is not corrected for **multiple comparisons**. After all, we are performing thousands of `t-tests` here (one for each voxel). A more conservative solution is to control the **family wise error** rate, i.e. the probability of making ony one false detection, say at `5%`. For that we use the so-called `Bonferroni correction`.


In [ ]:
_, threshold = threshold_stats_img(z_map, alpha = .05, height_control = 'bonferroni')

print('Bonferroni-corrected, p<.05 threshold: %.3f' % threshold)

plot_stat_map(z_map, bg_img = anat[0], threshold = threshold,
              display_mode = 'ortho', black_bg = True,
              title = 'Faces > Scrambled  (p<.05, FWE corrected)')
plt.show()

This is quite conservative.

#### False Discovery Rate (FDR) correction

A popular alternative is to control the **false discovery rate**, i.e. the expected proportion of false discoveries among detections. This is called the false disovery rate. `height_control='fdr`.

In [ ]:
_, threshold = threshold_stats_img(z_map, alpha = .05, height_control = 'fdr')

print('FDR, p<.05 threshold: %.3f' % threshold)

plot_stat_map(z_map, bg_img = anat[0], threshold = threshold,
              display_mode = 'ortho', black_bg = True,
              title = 'Faces > Scrambled  (p<.05, FDR corrected)')
plt.show()

#### Cluster threshold

It's a common practice to discard isolated voxels from the images. It is possible to generate a thresholded map with small clusters removed by providing a `cluster_threshold` argument. Here clusters smaller than `20` voxels will be discarded from the `FDR` corrected map.

In [ ]:
cluster_map, threshold = threshold_stats_img(
    z_map, alpha = .05, height_control='fdr', cluster_threshold = 20)

plot_stat_map(cluster_map, bg_img=anat[0], threshold=threshold,
              display_mode='ortho', black_bg=True, colorbar=False,
              title='Faces > Scrambled (p<.05, FDR corrected, k=20')
plt.show()

### Maps for all contrasts

In [ ]:
print('Computing contrasts')
import warnings
from nilearn import plotting
from nilearn.reporting import get_clusters_table

# Avoid getting warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    # Iterate on contrasts
    for contrast_id in contrast_list[0].keys():
        print("Contrast: %s" % contrast_id)
        # compute the contrasts
        z_map = fmri_glm.compute_contrast(
            [c[contrast_id] for c in contrast_list], output_type='z_score')
        # get threshold
        cluster_map, threshold = threshold_stats_img(z_map, alpha=.05, height_control='bonferroni', cluster_threshold=10)
        # get peak clusters    
        peaks = get_clusters_table(z_map, stat_threshold=threshold,
                               cluster_threshold=20)
        if len(peaks) != 0: 
            peak_xyz = peaks.loc[0, ['X', 'Y','Z']]
            # plot
            plotting.plot_stat_map(
                cluster_map, 
                bg_img = anat[0], 
                threshold = threshold, 
                display_mode='ortho',
                cut_coords = peak_xyz, 
                black_bg = True, 
                title = contrast_id + ' FWE p<.05, k=20')
            plotting.show()
        else:
            print('\t', contrast_id, 'has no significant voxels.')
    

### Let's visually compare different models

Let's compare our original model with some other possible models to see how different parameters affect our results. 

**Note:** This is not to fish for results that most match our hypothesis!

Some examples we could explore,
* a model without slice_time_ref adjusted
* a model wiht different confounds
* a model with different noise models
* a model with a different hrf model (contrasts might need adjusting!)

***If you are doing this in the interactive online notebook and kernel keeps dying, you could include only part of the subject's runs in the analysis. In the example below, I only include first 3 runs.***

In [ ]:
fmri_glm = FirstLevelModel(
    t_r = TR,
    slice_time_ref = 0,
    hrf_model = 'spm',
    drift_model = 'Cosine',
    high_pass = 1./128,
    smoothing_fwhm = 4,
    noise_model = 'ar1', 
    minimize_memory = False
)
fmri_glm = fmri_glm.fit(bold[:3], events[:3], confounds_glm[:3])
# ignore warnings about unused events columns
warnings.resetwarnings()
warnings.simplefilter('ignore')
##
design_matrices = fmri_glm.design_matrices_
plot_design_matrix(design_matrices[0], output_file=None)
fig = plt.gcf()
fig.set_size_inches(8,2)
plt.show()

In [ ]:
contrast_list = []
for design_matrix in design_matrices:
    """A small routine to append zeros in contrast vectors"""
    n_columns = design_matrix.shape[1] #number of predictors in our model
    def pad_vector(contrast_, n_columns):    
        return np.hstack((contrast_, np.zeros(n_columns - len(contrast_))))
    
    contrasts = {'Faces_Scrambled': pad_vector([1, 0, -2, 1], n_columns)}
    contrast_list.append(contrasts)

z_map = fmri_glm.compute_contrast([c['Faces_Scrambled'] for c in contrast_list],
                                      output_type='z_score')
    
_, threshold = threshold_stats_img(z_map, alpha=.001, height_control='fpr')
    
plot_stat_map(z_map, bg_img=anat[0], threshold=threshold,
                  display_mode='ortho', black_bg=True,
                  title='Faces > Scrambled (unc. > .001)')
plt.show()

Try some other models yourself! Refer to the [in-class notebook](https://nbviewer.org/github/dcdace/COGNESTIC-fMRI/blob/master/class-materials/04_nb_Subject-Level-Analysis.ipynb?flush_cache=true#The-impact-of-first-level-model-parameters) for help if needed. 